<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_ask_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-6.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-6.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heels hurt
2,4,black and blue marks
3,5,fast and slow heartbeat
4,5,increased heart rate


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-6.test.csv",header=None)
test.head()

,0,1
0,2,left heel pain
1,5,tachycardia
2,8,ARTHRITIS
3,8,arthrites
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'pubmed_ask.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'muscle',
 'a',
 'diclofenac',
 'was',
 'with',
 'were',
 'severe',
 'for',
 'myalgia',
 'lower',
 'my',
 'fatigue',
 'loss',
 'limb',
 'cramp',
 'weakness',
 'sodium',
 'by',
 'on',
 'stomach',
 '-',
 'depression',
 'back',
 'is',
 'at',
 'cramps',
 'patients',
 'that',
 'as',
 'upper',
 'gas',
 'joint',
 'or',
 'leg',
 'drug',
 'memory',
 'mg',
 'legs',
 'pains',
 'knee',
 'headache',
 'from',
 'shoulder',
 'unable',
 'group',
 'not',
 'arthralgia',
 'gastrointestinal',
 'treatment',
 'study',
 'all',
 'excessive',
 'neck',
 'this',
 'foot',
 'arthritis',
 'aches',
 'after',
 'numbness',
 'hip',
 'voltaren',
 'insomnia',
 'arthrotec',
 'disease',
 'nausea',
 'release',
 'p',
 'bleeding',
 '/',
 'increased',
 'asthenia',
 'both',
 'walk',
 'abdominal',
 'an',
 'heart',
 'tired',
 'swelling',
 'like',
 'dizziness',
 'skin',
 'no',
 'walkin

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.784018,3.387293,0.458259,28:24


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.426994,3.330547,0.463102,28:17


In [25]:
learn.save_encoder('pubmed_ask_fold6_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_ask_fold6_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.019363,3.681502,0.359626,01:14


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.875278,2.947713,0.487968,01:15


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.314969,2.484807,0.570856,03:21


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.013097,2.174501,0.618984,03:10
1,1.741751,2.019829,0.647059,03:16


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.556424,1.838748,0.681818,03:05
1,1.365404,1.748541,0.696524,03:15


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.213864,1.556151,0.725936,03:13
1,1.083843,1.486471,0.732620,03:13


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.024454,1.400401,0.744652,03:17
1,0.864913,1.327039,0.755348,03:18


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.842242,1.311477,0.762032,03:26
1,0.756831,1.255194,0.767380,03:14


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.704928,1.223510,0.778075,03:27
1,0.706269,1.233299,0.766043,03:16
2,0.566305,1.129396,0.792781,03:23
3,0.498338,1.101517,0.791444,03:15


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.505782,1.118047,0.787433,03:21
1,0.527525,1.070783,0.788770,03:25
2,0.450071,1.077508,0.815508,03:16
3,0.382608,1.044870,0.811497,03:26


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.398720,1.082238,0.807487,03:11
1,0.422349,1.106424,0.811497,03:26
2,0.358505,1.084593,0.823529,03:26
3,0.307162,1.046903,0.826203,03:23


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.308522,1.062109,0.818182,03:32
1,0.329750,1.084863,0.804813,03:31
2,0.308079,1.072810,0.823529,03:11
3,0.254675,1.060218,0.818182,03:19


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.255687,1.086500,0.822192,03:19
1,0.329694,1.084687,0.816845,03:18
2,0.263176,1.053213,0.820856,03:13
3,0.212667,0.999137,0.820856,03:20


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.240769,1.112296,0.818182,03:25
1,0.265858,1.078119,0.824866,03:26
2,0.250316,1.065041,0.827540,03:20
3,0.224983,1.079978,0.828877,03:28


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.215013,1.052123,0.826203,03:15
1,0.249723,1.118284,0.819519,03:13
2,0.211082,1.141509,0.835562,03:27
3,0.180234,1.084579,0.835562,03:12


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.226840,1.179665,0.822192,03:36
1,0.199921,1.180077,0.822192,03:38


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.238248,1.243653,0.815508,03:17
1,0.195370,1.156435,0.830214,03:18


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.205385,1.160886,0.826203,03:21


In [44]:
# save the best model

learn.save_encoder('pubmed_ask_mixed_fold6')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 983, 8, 1026, 8, 8, 8, 8, 8, 8, 8, 102, 588, 27, 31, 34, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 745, 41, 41, 41, 426, 49, 10, 49, 49, 100, 100, 100, 100, 100, 51, 53, 53, 53, 53, 53, 842, 56, 58, 58, 577, 60, 64, 67, 71, 71, 76, 76, 864, 182, 79, 265, 265, 265, 265, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 289, 79, 79, 79, 79, 265, 80, 80, 80, 80, 80, 80, 81, 491, 83, 89, 90, 91, 91, 91, 91, 91, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 96, 96, 226, 100, 100, 100, 100, 100, 100, 230, 106, 108, 108, 108, 108, 788, 462, 111, 111, 111, 112, 113, 114, 398, 117, 117, 471, 983, 109, 122, 4, 125, 1025, 125, 125, 125, 125, 125, 125, 125, 772, 741, 319, 135, 89, 970, 139, 151, 151, 156, 157, 157, 163, 178, 171, 172, 172, 172, 172, 172, 173, 71, 179, 182, 182, 182, 183, 183, 183, 183, 184, 179, 187, 187, 695, 189, 190, 190, 191, 197, 197, 202, 205, 210, 210, 211, 767, 214, 214, 214, 896, 896, 896, 214, 214, 214, 58, 214, 2

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

653
0.754041570438799
